# Ch37 Homework (Simulation)

This homework uses disk.pyto familiarize you with how a modern hard drive works. It has a lot of different options, and unlike most of the other simulations, has a graphical animator to show you exactly what happens when the disk is in action. See the README for details.

## 1. Compute seek rotation, transfer times

Compute the seek, rotation, and transfer times for the following sets of requests: -a 0, -a 6, -a 30, -a 7,30,8, and finally -a 10,11,12,13.

command | Seek | Rotate | Transfer
--------|------|--------|-------
-a 0|0|165|30
-a 6|0|345|30
-a 30|80|265|30
-a 7,30,8|160|545|90
-a 10,11,12,13|40|425|120


## 2. Compute with new seek rate

Do the same requests above, but change the seek rate to different values: -S 2, -S 4, -S 8, -S 10, -S 40, -S 0.1. How do the times change?

command | Seek | Rotate | Transfer
--------|------|--------|-------
-S 2|60|705|150
-S 4|30|735|150
-S 8|15|750|150
-S 10|12|753|150
-S 40|3|762|150
-S 0.1|401|642|150

Based on our table, it appears as S is given a larger parameter, seek time goes down proportionally to how rotate time goes up. 


## 3. Compute with new rotation rate

Do the same requests above, but change the rotation rate: -R 0.1, -R 0.5, -R 0.01. How do the times change?

command | Seek | Rotate | Transfer
--------|------|--------|-------
-R 0.1|120|7528|1502
-R 0.5|120|1410|300
-R 0.01|120|76381|15001

Based on our table, it appears as R is given a smaller parameter, rotate time goes up proportionally to how transfer time goes up. Seek time stays the same.

## 4. Run SSTF (Shortest Seek Time First)

FIFO is not always best, e.g., with the request stream -a 7,30,8, what order should the requests be processed in? Run the shortest seek-time first (SSTF) scheduler (-p SSTF) on this workload; how long should it take (seek, rotation, transfer) for each request to be served?

The requests should be processed in the order 7, 8, 30. 

request|Seek|Rotate|Transfer
-------|----|------|--------
7|0|15|30
8|0|0|30
30|80|190|30

## 5. Run SATF (Shortest Access Time First)

Now use the shortest access-time first (SATF) scheduler (-p SATF). Does it make any difference for -a 7,30,8 workload? Find a set of requests where SATF outperforms SSTF; more generally, when is SATF better than SSTF?

Running SATF does not make any difference for our example of -a 7,30,8 since in this case shortest access time is the same as shortest seek time. A set of requests where SATF would outperform SSTF would be {30,17,4}, SATF is generally better when we have a faster rotation speed and slower seek time. 

## 6. Testing Skew

Here is a request stream to try: -a 10,11,12,13. What goes poorly when it runs? Try adding track skew to address this problem (-o skew). Given the default seek rate, what should the skew be to maximize performance? What about for different seek rates (e.g., -S 2, -S 4)? In general, could you write a formula to figure out the skew?

When we run that sequence, we are seeing a long rotation between 11 and 12 that we dont need since they're sequential. Skew of 2 is the optimal value to maximize performance. Skew of 1 is better with different seek rates and it appears the skew should be $ skew = int(\frac{2}{r_s}) $

## 7. Find the Bandwidth

Specify a disk with different density per zone, e.g., -z 10,20,30, which specifies the angular difference between blocks on the outer, middle, and inner tracks. Run some random requests (e.g., -a -1 -A 5,-1,0, which specifies that random requests should be used via the -a -1flag and that five requests ranging from 0 to the max be generated), and compute the seek, rotation, and transfer times. Use different random seeds. What is the bandwidth (in sectors per unit time) on the outer, middle, and inner tracks?

Seek is 40 max, 
rotation is maximum 355, 
bandwidth appears to be 20 on outer, 10 on middle, 10 on inner as transfer rates 

## 8. Scheduling Window tests

A scheduling window determines how many requests the disk can examine at once. Generate random workloads (e.g., -A 1000,-1,0, with different seeds) and see how long the SATF scheduler takes when the scheduling window is changed from 1 up to the number of requests. How big of a window is needed to maximize performance? Hint: use the -c flag and don’t turn on graphics (-G) to run these quickly. When the scheduling window is set to 1, does it matter which policy you are using?

SATF works fast and cuts time significantly from 7. It does not matter when window is 1. 

## 9. SATF vs Bounded SATF for starvation

Create a series of requests to starve a particular request, assuming an SATF policy. Given that sequence, how does it perform if you use a bounded SATF (BSATF) scheduling approach? In this approach, you specify the scheduling window (e.g., -w 4); the scheduler only moves onto the next window of requests when all requests in the current window have been serviced. Does this solve starvation? How does it perform, as compared to SATF? In general, how should a disk make this trade-off between performance and starvation avoidance?

This does not solve starvation and it performs much worse than SATF. 

## 10. When is greedy not optimal

All the scheduling policies we have looked at thus far are greedy; they pick the next best option instead of looking for an optimal schedule. Can you find a set of requests in which greedy is not optimal?

Any set where we have data continuously coming in would create starvation with a greedy algorithm which is the main issue here. 